In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
print("Tensorflow version: ", tf.__version__)
print("Numpy version: ", np.__version__)
print("Pandas version: ", pd.__version__)
print("Scikit-learn version: ", sklearn.__version__)

In [ ]:
# Clone the repo and run current notebook.
file = np.load('/content/drive/MyDrive/wavelet_CWRU_data.npz')
print(file.files)

In [ ]:
data = file['data']
labels = file['labels']
print(data.shape, labels.shape)

In [ ]:
print(data[0])
print(labels)

In [ ]:
category_labels = np.unique(labels)
print(category_labels)

In [ ]:
labels = pd.Categorical(labels, categories = category_labels).codes

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = 1000, random_state = 829,
                                                                   stratify = labels)
# reshape data as input should be three dimensional to a CNN

train_data = train_data.reshape(len(train_data),32,32,1)
test_data = test_data.reshape(len(test_data),32,32,1)

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Shuffle data
index = np.random.permutation(len(train_labels))
trian_data, trian_labels = train_data[index], train_labels[index]

print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape )

In [ ]:
demo_model = Sequential([
    layers.Conv2D(32,3,activation= 'relu', input_shape = (32,32,1)),
    layers.MaxPool2D(2),
    layers.Conv2D(32,4,activation = 'relu'),
    layers.MaxPool2D(2),
#    layers.LSTM(32, activation = "tanh", return_sequences = True),
    #layers.LSTM(32, activation = "tanh"),
    layers.Flatten(),
    layers.Dense(128,activation = 'relu'),
    layers.Dense(96, activation = 'relu'),
    layers.Dense(10 ,activation ='softmax')
])
demo_model.summary()

In [ ]:
def create_compiled_model():
    model = Sequential([
    layers.Conv2D(32,3,activation= 'relu', input_shape = (32,32,1)),
    layers.MaxPool2D(2),
    layers.Conv2D(32,4,activation = 'relu'),
    layers.MaxPool2D(2),
    #layers.LSTM(32, activation = "tanh", return_sequences = True),
#    layers.LSTM(32, activation = "tanh"),
    layers.Flatten(),
    layers.Dense(128,activation = 'relu'),
    layers.Dense(96, activation = 'relu'),
    layers.Dense(10,activation= 'softmax')
    ])
    model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001), 
                  metrics= ['accuracy'])
    return model

In [ ]:
res = np.empty(10)
res[:] = np.nan
for i in range(10):
    model = create_compiled_model()
    history = model.fit(train_data, train_labels, epochs = 100, batch_size = 128, verbose = 0)    # Verbosity is set to zero
    res[i] = model.evaluate(test_data, test_labels, batch_size = 128, verbose = 0)[1]            # Verbosity is set to zero
    print('Loop iteration %d, Accuracy: %4.4f' % (i+1, res[i]))
    if res[i]>=np.max(res[:(i+1)]):
        best_model = model

In [ ]:
print('Average accuracy:%4.4f'%(np.mean(res))) # After running the model 10 times
print("Best accuracy: %4.4f"%(np.max(res)))
print("Worst accuracy: %4.4f"%(np.min(res)))
print('Standard deviation: %4.4f' % (np.std(res)))

In [ ]:
prediction = best_model.predict(test_data)
prediction_labels = list([])
for each_prediction in prediction:
    prediction_labels.append(list(each_prediction).index(max(each_prediction)))
from sklearn.metrics import confusion_matrix
true_labels = np.argmax(test_labels, axis = 1)
matrix = confusion_matrix(true_labels, prediction_labels)
import seaborn as sns
matrix = pd.DataFrame(matrix)
plt.figure()
sns.heatmap(matrix, annot= True, fmt = "d",
           xticklabels = category_labels,
           yticklabels = category_labels, cmap = "Blues", cbar = False)
plt.xticks(rotation = 90)
plt.show()